# 🛠️ PPE Detection – Notebook v2
Cette version intègre :
- une configuration **centralisée** et dynamique (chemins adap­tables)
- un _fine‑tuning_ YOLOv11 avec **data‑augmentation** + **early‑stopping**
- la recherche **automatique** du meilleur poids (`best.pt`)
- une fonction **`quick_metrics()`** qui extrait en une ligne les principales métriques Ultralytics
- des helpers pour détection / filtrage / affichage fiables (pas de *KeyError*)

In [2]:
#!/usr/bin/env python
import os, shutil, glob, random, json, yaml
from pathlib import Path
import numpy as np, pandas as pd, matplotlib.pyplot as plt, cv2, torch
from ultralytics import YOLO
print(f'Torch : {torch.__version__} | CUDA dispo : {torch.cuda.is_available()}')

Torch : 2.7.1+cu118 | CUDA dispo : True


In [3]:
class CFG:
    # --- Chemins (à adapter) ---
    ROOT = Path('/home/maxime/DataDevIA/computervision/security-checker')  # dossier projet
    DATA_DIR = Path('/home/maxime/DataDevIA/computervision/datasetcomputervision/css-data')
    WORKING = ROOT  # on garde tout dans ROOT pour simplifier

    # fichiers clé
    BASE_WEIGHTS = ROOT / 'yolo11n.pt'        # poids COCO d'origine
    DATA_YAML    = ROOT / 'ppe.yaml'          # sera (re)créé si besoin

    # hyper‑params
    EPOCHS      = 30
    IMG_SIZE    = 640
    BATCH       = 12
    DEVICE      = 'cuda' if torch.cuda.is_available() else 'cpu'

    # classes PPE (index ↔ nom)
    PPE_CLASSES = {
        0: 'Hardhat', 1: 'Mask', 2: 'NO-Hardhat', 3: 'NO-Mask',
        4: 'NO-Safety Vest', 5: 'Person', 6: 'Safety Cone',
        7: 'Safety Vest', 8: 'Machinery', 9: 'Vehicle'
    }

In [4]:
def quick_metrics(metrics_obj):
    """Affiche precision / recall / mAP50 / mAP50‑95 d'un objet DetMetrics."""
    res = metrics_obj.results_dict
    print('\n📊 Quick metrics')
    print(f"Precision   : {res['metrics/precision(B)']:.3f}")
    print(f"Recall      : {res['metrics/recall(B)']:.3f}")
    print(f"mAP50       : {res['metrics/mAP50(B)']:.3f}")
    print(f"mAP50‑95    : {res['metrics/mAP50-95(B)']:.3f}")

In [5]:
def clean_runs():
    runs_dir = CFG.WORKING / 'runs'
    if runs_dir.exists():
        print('♻️ Suppression de', runs_dir)
        shutil.rmtree(runs_dir)
    else:
        print('runs/ déjà vide')

In [ ]:
def ensure_yaml():
    if not CFG.DATA_YAML.exists():
        data = {
            'path': str(CFG.DATA_DIR),
            'train': 'train/images',
            'val'  : 'valid/images',
            'test' : 'test/images',
            'names': list(CFG.PPE_CLASSES.values())
        }
        with open(CFG.DATA_YAML, 'w') as f:
            yaml.safe_dump(data, f)
        print('✅ ppe.yaml créé ->', CFG.DATA_YAML)
    else:
        print('ppe.yaml trouvé ->', CFG.DATA_YAML)

ensure_yaml()

ppe.yaml trouvé -> /home/maxime/DataDevIA/computervision/security-checker/ppe.yaml


## 🚂 Fine‑tuning du modèle PPE

In [7]:
# Nettoyer d’éventuels anciennes runs
clean_runs()

# Charger le modèle de base (COCO)
base_model = YOLO(str(CFG.BASE_WEIGHTS))

# Lancer l'entraînement (revenir à GPU si disponible)
base_model.train(
    data=str(CFG.DATA_YAML),
    epochs=CFG.EPOCHS,
    imgsz=CFG.IMG_SIZE,
    batch=CFG.BATCH,
    device=CFG.DEVICE,  # <--- utiliser CFG.DEVICE (par ex. "0")
    project=str(CFG.WORKING / 'runs' / 'detect'),
    name='ppe_train',
    patience=10,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    flipud=0.2, fliplr=0.5,
    mosaic=0.8, mixup=0.2
)



♻️ Suppression de /home/maxime/DataDevIA/computervision/security-checker/runs
Ultralytics 8.3.151 🚀 Python-3.12.3 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5931MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=12, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/maxime/DataDevIA/computervision/security-checker/ppe.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.2, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=/home/maxime/DataDevIA/computervision/security-checker/yolo11n.pt, momentum=0.937, 

train: Scanning /home/maxime/DataDevIA/computervision/datasetcomputervision/css-data/train/labels.cache... 2605 images, 6 backgrounds, 0 corrupt: 100%|██████████| 2605/2605 [00:00<?, ?it/s]

train: /home/maxime/DataDevIA/computervision/datasetcomputervision/css-data/train/images/004720_jpg.rf.afc486560a4004c7cfd67910af31a29c.jpg: 1 duplicate labels removed
train: /home/maxime/DataDevIA/computervision/datasetcomputervision/css-data/train/images/construction-813-_jpg.rf.b085952261fd98f2e76b8065de149b5f.jpg: 1 duplicate labels removed


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 162.7±41.5 MB/s, size: 52.1 KB)


val: Scanning /home/maxime/DataDevIA/computervision/datasetcomputervision/css-data/valid/labels.cache... 114 images, 10 backgrounds, 0 corrupt: 100%|██████████| 114/114 [00:00<?, ?it/s]


...

      30/30      2.36G      1.004      0.946      1.222         11        640: 100%|██████████| 218/218 [00:21<00:00, 10.02it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  9.16it/s]


                   all        114        697      0.822      0.664      0.733      0.386

30 epochs completed in 0.196 hours.
Optimizer stripped from /home/maxime/DataDevIA/computervision/security-checker/runs/detect/ppe_train/weights/last.pt, 5.5MB
Optimizer stripped from /home/maxime/DataDevIA/computervision/security-checker/runs/detect/ppe_train/weights/best.pt, 5.5MB

Validating /home/maxime/DataDevIA/computervision/security-checker/runs/detect/ppe_train/weights/best.pt...
Ultralytics 8.3.151 🚀 Python-3.12.3 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5931MiB)
YOLO11n summary (fused): 100 layers, 2,584,102 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  3.48it/s]


                   all        114        697      0.819      0.665      0.724      0.393
               Hardhat         42         79      0.965      0.747      0.832      0.519
                  Mask         19         21      0.978      0.857      0.873      0.494
            NO-Hardhat         37         69      0.816      0.507      0.611      0.278
               NO-Mask         44         74      0.738      0.459      0.526      0.226
        NO-Safety Vest         56        106      0.812      0.538       0.63      0.312
                Person         84        166      0.825      0.663      0.768      0.449
           Safety Cone         13         44      0.807      0.818      0.863      0.439
           Safety Vest         28         41      0.938      0.743       0.82      0.441
             Machinery         26         55      0.777      0.818      0.854      0.493
               Vehicle         16         42      0.536        0.5      0.466      0.282
Speed: 0.3ms preproce

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7306c90bbe60>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0

In [9]:
from pathlib import Path
import glob
from ultralytics import YOLO

# 1. Chemin vers le .pt “best” issu de l’entraînement PPE
best_pt_path = "/home/maxime/DataDevIA/computervision/security-checker/runs/detect/ppe_train/weights/best.pt"

# 2. Charger le modèle PPE (sans device ici)
model_ppe = YOLO(best_pt_path)
print("✅ Modèle PPE chargé :", best_pt_path)

# 3. Déplacer tous les poids du modèle sur le CPU
model_ppe.model.cpu()

# 4. Définir le dossier test/images et lister toutes les .jpg
data_dir    = Path("/home/maxime/DataDevIA/computervision/datasetcomputervision/css-data")
test_img_dir = data_dir / "test" / "images"
test_imgs    = glob.glob(str(test_img_dir / "*.jpg"))
print(f"✅ {len(test_imgs)} images trouvées dans « {test_img_dir} »")

# 5. Créer (si besoin) le dossier de sortie predict_ppe
output_dir = Path("/home/maxime/DataDevIA/computervision/security-checker") / "runs" / "detect" / "predict_ppe"
output_dir.mkdir(parents=True, exist_ok=True)
print("→ Les prédictions seront enregistrées sous :", output_dir)

# 6. Lancer la prédiction sur CPU
model_ppe.predict(
    source=test_imgs,               # liste de chemins .jpg
    device="cpu",                   # forcer CPU
    save=True,
    project=str(Path("/home/maxime/DataDevIA/computervision/security-checker") / "runs" / "detect"),
    name="predict_ppe",
    exist_ok=True
)

print("✅ Prédictions sauvegardées dans :", output_dir)




✅ Modèle PPE chargé : /home/maxime/DataDevIA/computervision/security-checker/runs/detect/ppe_train/weights/best.pt
✅ 82 images trouvées dans « /home/maxime/DataDevIA/computervision/datasetcomputervision/css-data/test/images »
→ Les prédictions seront enregistrées sous : /home/maxime/DataDevIA/computervision/security-checker/runs/detect/predict_ppe

0: 640x640 1 Person, 100.3ms
1: 640x640 3 Hardhats, 4 NO-Masks, 3 NO-Safety Vests, 4 Persons, 100.3ms
2: 640x640 1 Person, 1 Safety Vest, 100.3ms
3: 640x640 2 Hardhats, 1 NO-Safety Vest, 1 Person, 100.3ms
4: 640x640 (no detections), 100.3ms
5: 640x640 2 Hardhats, 2 NO-Masks, 4 Persons, 100.3ms
6: 640x640 1 Mask, 1 NO-Hardhat, 1 NO-Safety Vest, 1 Person, 100.3ms
7: 640x640 1 Hardhat, 1 NO-Safety Vest, 1 Person, 15 Vehicles, 100.3ms
8: 640x640 1 Hardhat, 1 Person, 1 Safety Vest, 100.3ms
9: 640x640 1 Hardhat, 1 Person, 2 Machinerys, 1 Vehicle, 100.3ms
10: 640x640 1 Mask, 1 NO-Hardhat, 1 NO-Safety Vest, 1 Person, 100.3ms
11: 640x640 2 Hardhats, 

In [10]:
# Relancer la validation, cette fois en CPU ou avec workers=0 si nécessaire
metrics = model_ppe.val(
    data=str(CFG.DATA_YAML),
    imgsz=CFG.IMG_SIZE,
    save=True,
    device="cpu",     # ou workers=0 si vous souhaitez rester en GPU
)

# quick_metrics les affiche déjà, mais on peut aussi extraire le dictionnaire complet :
md = metrics.results_dict
print("\n→ Résultats globaux :")
print(f"  • Precision : {md['metrics/precision(B)']:.3f}")
print(f"  • Recall    : {md['metrics/recall(B)']:.3f}")
print(f"  • mAP50     : {md['metrics/mAP50(B)']:.3f}")
print(f"  • mAP50-95  : {md['metrics/mAP50-95(B)']:.3f}")

print("\n→ mAP50 par classe :")
for idx, cls_name in CFG.PPE_CLASSES.items():
    print(f"  • {cls_name:<13} : {metrics.maps[idx]:.3f}")


Ultralytics 8.3.151 🚀 Python-3.12.3 torch-2.7.1+cu118 CPU (AMD Ryzen 7 5800H with Radeon Graphics)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 262.3±87.5 MB/s, size: 59.5 KB)


val: Scanning /home/maxime/DataDevIA/computervision/datasetcomputervision/css-data/valid/labels.cache... 114 images, 10 backgrounds, 0 corrupt: 100%|██████████| 114/114 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:12<00:00,  1.51s/it]


                   all        114        697      0.814      0.668      0.725      0.394
               Hardhat         42         79      0.953      0.747      0.832      0.521
                  Mask         19         21      0.977      0.857      0.873      0.494
            NO-Hardhat         37         69      0.792      0.507      0.609      0.277
               NO-Mask         44         74      0.732      0.459      0.525      0.224
        NO-Safety Vest         56        106      0.805      0.538      0.626      0.311
                Person         84        166       0.81      0.663      0.769       0.45
           Safety Cone         13         44      0.805      0.818      0.863       0.44
           Safety Vest         28         41      0.939      0.745       0.82      0.443
             Machinery         26         55       0.77      0.818      0.854      0.496
               Vehicle         16         42      0.554      0.524      0.484      0.289
Speed: 3.5ms preproce

In [11]:
import glob, IPython.display as disp

val_dir = Path(CFG.WORKING) / "runs" / "detect" / "val"
all_png = glob.glob(str(val_dir / "*" / "*.png"))

for img_path in sorted(all_png):
    print("📊 Affichage :", img_path.split("/")[-1])
    disp.display(disp.Image(img_path))



In [12]:
from pathlib import Path
import glob

# 1. Pointage vers le dossier test/images
data_dir     = Path("/home/maxime/DataDevIA/computervision/datasetcomputervision/css-data")
test_img_dir = data_dir / "test" / "images"

# 2. Liste explicite de tous les .jpg
test_pattern = str(test_img_dir / "*.jpg")
test_imgs    = glob.glob(test_pattern)
print(f"✅ {len(test_imgs)} images trouvées dans «{test_pattern}»")

# 3. (Re)crée le dossier de sortie runs/detect/predict_ppe
output_dir = Path("/home/maxime/DataDevIA/computervision/security-checker") \
                 / "runs" / "detect" / "predict_ppe"
output_dir.mkdir(parents=True, exist_ok=True)
print("→ Les prédictions seront enregistrées sous :", output_dir)

# 4. Assurez-vous que `model_ppe` est chargé et placé sur CPU en mode éval
#    (on suppose que model_ppe a été instancié plus haut, par ex. :
#     model_ppe = YOLO("/home/maxime/.../runs/detect/ppe_train/weights/best.pt") )
model_ppe.model.to("cpu")
model_ppe.model.eval()

# 5. Lancez la prédiction sur CPU
model_ppe.predict(
    source=test_imgs,       # liste explicite de chemins .jpg
    device="cpu",           # forcer CPU (évite pin_memory/GPU)
    save=True,
    project=str(Path("/home/maxime/DataDevIA/computervision/security-checker") / "runs" / "detect"),
    name="predict_ppe",
    exist_ok=True
)

print("✅ Prédictions sauvegardées dans :", output_dir)




✅ 82 images trouvées dans «/home/maxime/DataDevIA/computervision/datasetcomputervision/css-data/test/images/*.jpg»
→ Les prédictions seront enregistrées sous : /home/maxime/DataDevIA/computervision/security-checker/runs/detect/predict_ppe

0: 640x640 1 Person, 97.8ms
1: 640x640 3 Hardhats, 4 NO-Masks, 3 NO-Safety Vests, 4 Persons, 97.8ms
2: 640x640 1 Person, 1 Safety Vest, 97.8ms
3: 640x640 2 Hardhats, 1 NO-Safety Vest, 1 Person, 97.8ms
4: 640x640 (no detections), 97.8ms
5: 640x640 2 Hardhats, 2 NO-Masks, 4 Persons, 97.8ms
6: 640x640 1 Mask, 1 NO-Hardhat, 1 NO-Safety Vest, 1 Person, 97.8ms
7: 640x640 1 Hardhat, 1 NO-Safety Vest, 1 Person, 15 Vehicles, 97.8ms
8: 640x640 1 Hardhat, 1 Person, 1 Safety Vest, 97.8ms
9: 640x640 1 Hardhat, 1 Person, 2 Machinerys, 1 Vehicle, 97.8ms
10: 640x640 1 Mask, 1 NO-Hardhat, 1 NO-Safety Vest, 1 Person, 97.8ms
11: 640x640 2 Hardhats, 2 NO-Masks, 2 NO-Safety Vests, 2 Persons, 1 Safety Vest, 97.8ms
12: 640x640 7 Persons, 6 Machinerys, 9 Vehicles, 97.8ms
13

In [13]:
def find_best_pt():
    pattern = CFG.WORKING / 'runs' / 'detect' / '**' / 'weights' / 'best.pt'
    best_candidates = list(glob.glob(str(pattern), recursive=True))
    if not best_candidates:
        raise FileNotFoundError('Aucun best.pt trouvé !')
    best_pt = max(best_candidates, key=os.path.getmtime)
    print('✨ best.pt sélectionné ->', best_pt)
    return Path(best_pt)

BEST_PT = find_best_pt()

✨ best.pt sélectionné -> /home/maxime/DataDevIA/computervision/security-checker/runs/detect/ppe_train/weights/best.pt


In [14]:
test_pattern = str(CFG.DATA_DIR / "test" / "images" / "*.jpg")
output_dir = CFG.WORKING / "runs" / "detect" / "predict_ppe"

# On force le predict() à tourner sur CPU, pour éviter le bug pin_memory/GPU
model_ppe.predict(
    source=test_pattern,
    device="cpu",              # <--- ici on force l’inférence sur CPU
    save=True,
    project=str(CFG.WORKING / "runs" / "detect"),
    name="predict_ppe",
    exist_ok=True
)

print("✅ Prédictions sauvegardées dans :", output_dir)




image 1/82 /home/maxime/DataDevIA/computervision/datasetcomputervision/css-data/test/images/-4405-_png_jpg.rf.82b5c10b2acd1cfaa24259ada8e599fe.jpg: 640x640 2 Machinerys, 72.3ms
image 2/82 /home/maxime/DataDevIA/computervision/datasetcomputervision/css-data/test/images/000005_jpg.rf.96e9379ccae638140c4a90fc4b700a2b.jpg: 640x640 2 Hardhats, 2 NO-Masks, 4 Persons, 52.6ms
image 3/82 /home/maxime/DataDevIA/computervision/datasetcomputervision/css-data/test/images/002551_jpg.rf.ce4b9f934161faa72c80dc6898d37b2d.jpg: 640x640 2 Hardhats, 2 NO-Safety Vests, 3 Persons, 73.5ms
image 4/82 /home/maxime/DataDevIA/computervision/datasetcomputervision/css-data/test/images/003357_jpg.rf.9867f91e88089bb68dc95947d5116d14.jpg: 640x640 3 Hardhats, 2 NO-Masks, 1 NO-Safety Vest, 1 Person, 1 Safety Cone, 56.0ms
image 5/82 /home/maxime/DataDevIA/computervision/datasetcomputervision/css-data/test/images/004063_jpg.rf.1b7cdc4035bcb24ef69b8798b444053e.jpg: 640x640 5 Hardhats, 3 NO-Safety Vests, 6 Persons, 3 Safet

In [15]:
import IPython.display as disp, glob
val_pngs = glob.glob(str(CFG.WORKING / 'runs' / 'detect' / 'val' / '*' / 'results.png'))
if val_pngs:
    disp.display(disp.Image(val_pngs[-1]))

### 🖼️ Fonctions de détection / filtrage

In [16]:
def detect_all_classes(image_paths, model=model_ppe):
    rows = []
    for p in image_paths:
        res = model(p)
        boxes = []
        classes = set()
        for r in res:
            for b in r.boxes:
                idx = int(b.cls)
                if idx not in CFG.PPE_CLASSES:
                    continue
                label = CFG.PPE_CLASSES[idx]
                boxes.append({'label': label, 'confidence': float(b.conf), 'coords': tuple(map(int, b.xyxy[0]))})
                classes.add(label)
        rows.append({'image_path': p, 'detected_boxes': boxes, 'detected_classes': sorted(classes)})
    return pd.DataFrame(rows)

def filter_images(df, allowed=None):
    """Garde uniquement les lignes dont detected_classes est sous‑ensemble de allowed (si allowed fourni)."""
    if allowed is None:
        return df
    allowed = set(allowed)
    return df[df['detected_classes'].apply(lambda cls: set(cls).issubset(allowed))]

In [27]:
# Vérifiez que le fichier existe et affichez la première image
import os
print(sample_imgs[:3])
print([os.path.exists(p) for p in sample_imgs[:3]])


['/home/maxime/DataDevIA/computervision/datasetcomputervision/css-data/valid/images/construction-277-_jpg.rf.bf73a4edc7a1f7d412bc3267cbafcda7.jpg', '/home/maxime/DataDevIA/computervision/datasetcomputervision/css-data/valid/images/autox4_mp4-72_jpg.rf.5e9c1836f027bf40eb8898e68f2efdeb.jpg', '/home/maxime/DataDevIA/computervision/datasetcomputervision/css-data/valid/images/IMG_3103_mp4-17_jpg.rf.3223d405c1b3657aa54e170e32c8fe52.jpg']
[True, True, True]


In [28]:
import glob, pandas as pd

# 1) Petit échantillon de 5 images de validation
sample_imgs = glob.glob(str(CFG.DATA_DIR / "valid" / "images" / "*.jpg"))[:5]
if not sample_imgs:
    raise ValueError("Aucune image trouvée dans le dossier de validation.")

# 2) Exécuter la détection – renvoie df_details avec colonnes ['image_path','detected_boxes','detected_classes']
df_details = detect_all_classes(sample_imgs)

# 3) Afficher un aperçu
print("Colonnes retournées :", list(df_details.columns))
display(df_details.head())

if df_details.empty:
    print("⚠️  Aucune détection retournée. Vérifiez que le modèle fonctionne ou essayez d’autres images.")
else:
    # 4) On a déjà une colonne 'detected_classes' par ligne-image
    df_summary = df_details[['image_path', 'detected_classes']].copy()
    display(df_summary)






image 1/1 /home/maxime/DataDevIA/computervision/datasetcomputervision/css-data/valid/images/construction-277-_jpg.rf.bf73a4edc7a1f7d412bc3267cbafcda7.jpg: 640x640 2 Machinerys, 62.0ms
Speed: 2.0ms preprocess, 62.0ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/maxime/DataDevIA/computervision/datasetcomputervision/css-data/valid/images/autox4_mp4-72_jpg.rf.5e9c1836f027bf40eb8898e68f2efdeb.jpg: 640x640 8 Safety Cones, 1 Vehicle, 61.6ms
Speed: 1.2ms preprocess, 61.6ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/maxime/DataDevIA/computervision/datasetcomputervision/css-data/valid/images/IMG_3103_mp4-17_jpg.rf.3223d405c1b3657aa54e170e32c8fe52.jpg: 640x640 1 Mask, 1 NO-Hardhat, 1 NO-Safety Vest, 1 Person, 78.0ms
Speed: 12.4ms preprocess, 78.0ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/maxime/DataDevIA/computervision/datasetcomputervision/css-data/valid/images/youtube-226_jpg.r

,image_path,detected_boxes,detected_classes
0,/home/maxime/DataDevIA/computervision/datasetc...,"[{'label': 'Machinery', 'confidence': 0.603937...",[Machinery]
1,/home/maxime/DataDevIA/computervision/datasetc...,"[{'label': 'Vehicle', 'confidence': 0.93421447...","[Safety Cone, Vehicle]"
2,/home/maxime/DataDevIA/computervision/datasetc...,"[{'label': 'Person', 'confidence': 0.958926856...","[Mask, NO-Hardhat, NO-Safety Vest, Person]"
3,/home/maxime/DataDevIA/computervision/datasetc...,"[{'label': 'Machinery', 'confidence': 0.836218...",[Machinery]
4,/home/maxime/DataDevIA/computervision/datasetc...,"[{'label': 'Person', 'confidence': 0.949202895...","[Mask, NO-Hardhat, NO-Mask, NO-Safety Vest, Pe..."


,image_path,detected_classes
0,/home/maxime/DataDevIA/computervision/datasetc...,[Machinery]
1,/home/maxime/DataDevIA/computervision/datasetc...,"[Safety Cone, Vehicle]"
2,/home/maxime/DataDevIA/computervision/datasetc...,"[Mask, NO-Hardhat, NO-Safety Vest, Person]"
3,/home/maxime/DataDevIA/computervision/datasetc...,[Machinery]
4,/home/maxime/DataDevIA/computervision/datasetc...,"[Mask, NO-Hardhat, NO-Mask, NO-Safety Vest, Pe..."
